In [27]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
import matplotlib.pyplot as plt
warnings.simplefilter('ignore')

In [45]:
# 아워박스 수동으로 긁은 데이터
# 위 데이터 기간 맞춰서 친스데이터 다운로드
ob_df = pd.read_csv('/Users/seanyu_mac/Downloads/아워박스_통합관리_조회_초안 - 시트1.csv')
fs_df = pd.read_excel('/Users/seanyu_mac/Downloads/주문내역_2025-01-01~2025-03-31.xlsx')
fs_df = fs_df[fs_df['주문 상태'].isin(['결제완료', '배송중', '배송준비', '배송완료', '반품진행(반품진행)', '구매확정', '반품진행(수거진행)', '반품완료(반품완료)'])]

In [47]:
# 두개 테이블의 주문 일시와 결제 완료일은 datetime 형식으로 변환 후 str 타입으로 변환
ob_df['주문일시'] = pd.to_datetime(ob_df['주문일시'])
ob_df = ob_df.sort_values(by='주문일시', ascending=False)
ob_df = ob_df.reset_index()
fs_df['결제 완료일'] = pd.to_datetime(fs_df['결제 완료일'])
fs_df = fs_df.sort_values(by='결제 완료일', ascending=False)
fs_df = fs_df.reset_index()

fs_df['결제 완료일'] = fs_df['결제 완료일'].dt.strftime('%Y-%m-%d %H:%M:%S')
ob_df['주문일시'] = ob_df['주문일시'].astype(str)

# 두 테이블의 우편번호를 str 타입으로 변환
ob_df['우편번호'] = ob_df['우편번호'].astype(str)
fs_df['우편번호'] = fs_df['우편번호'].fillna(0)
fs_df['우편번호'] = fs_df['우편번호'].astype(int)
fs_df['우편번호'] = fs_df['우편번호'].astype(str)
fs_df['우편번호']

# 두 테이블의 주문번호를 앞 5자리 추출 후 str 타입으로 변환
ob_df['주문번호'] = ob_df['주문번호'].apply(lambda x: x[:5])
fs_df['주문 번호'] = fs_df['주문 번호'].apply(lambda x: x[:5])

# 아워박스 데이터의 결제 금액을 뜻하는 sales 컬럼 생성 후 str 타입으로 변환
ob_df['sales'] = ob_df['수량'] * ob_df['판매단가']
ob_df['sales'] = ob_df['sales'].astype(str)
fs_df['정산가'] = fs_df['정산가'].astype(str)

# 머지하기 위한 key 컬럼을 생성 : 주문일시 + 우편번호 + 주문번호 + 결제금액
# ob_df['key'] = ob_df['주문일시'] + ob_df['우편번호'] + ob_df['주문번호'] + ob_df['sales']
ob_df['key'] = ob_df['주문일시'] + ob_df['우편번호'] + ob_df['주문번호'] + ob_df['sales']
# fs_df['key'] = fs_df['결제 완료일'] + fs_df['우편번호'] + fs_df['주문 번호'] + fs_df['정산가']
fs_df['key'] = fs_df['결제 완료일'] + fs_df['우편번호'] + fs_df['주문 번호'] + fs_df['정산가']

# 특이 케이스 삭제
fs_df.drop(fs_df[fs_df['key'] == '2025-03-14 19:07:1685995083821370'].index, inplace=True)
fs_df.drop(fs_df[fs_df['상품명'] == '[테스트 상품] 친한스토어 예시 상품'].index, inplace=True)
fs_df.drop(fs_df[fs_df['상품명'].str.contains('테스트')].index, inplace=True)



Index([935, 936, 937, 938, 949, 950, 952, 953, 954, 955, 956, 957, 959, 969], dtype='int64')

In [49]:
# 아워박스 채널상품명 필드 - 친한스토어 판매상품명  =  fs_df['상품명']
# 아워박스 채널상품ID 필드 - 친한스토어 상품코드  =  fs_df['상품 번호']
# 아워박스 옵션1 필드 - 친한스토어 옵션명  =  fs_df['옵션명']

types = []

for i in fs_df['상품명']:
    if '공동구매' in i:
        types.append('공동구매')
    elif '임직원' in i:
        types.append('임직원몰 상품')
    elif '스페셜' in i:
        types.append('친한구매')

fs_df['구분'] = types

In [51]:
temp_df = pd.merge(ob_df, fs_df[['key', '상품명', '상품 번호', '옵션명', '구분']], on='key', how='left')
datas = []

for i in temp_df['품목명']:
    if '>S' in i:
        data = i.split('>S')[1].strip()
    elif '>D' in i:
        data = i.split('>D')[1].strip()
    datas.append(data)

temp_df['품목명'] = datas

temp_df['품목명'] = temp_df['품목명'].str.replace('</span>', '')
temp_df['품목명'] = temp_df['품목명'].apply(lambda x: x.strip())

temp_df['도서산간여부'] = temp_df['도서산간여부'].str.replace("<span class='square square-gray'>", '')
temp_df['도서산간여부'] = temp_df['도서산간여부'].str.replace("</span>", '')
temp_df['도서산간여부'] = temp_df['도서산간여부'].apply(lambda x: x.strip())

temp_df['주문서상태'] = temp_df['주문서상태'].str.replace("<span class='square square-primary'>", '')
temp_df['주문서상태'] = temp_df['주문서상태'].str.replace("<span class='square square-warning'>", '')
temp_df['주문서상태'] = temp_df['주문서상태'].str.replace("</span>", '')
temp_df['주문서상태'] = temp_df['주문서상태'].apply(lambda x: x.strip())

temp_df['채널상품명'] = temp_df['상품명']
temp_df['채널아이디'] = temp_df['상품 번호']
temp_df['옵션1'] = temp_df['옵션명']

del temp_df['sales']
del temp_df['key']
del temp_df['상품명']
del temp_df['상품 번호']
del temp_df['옵션명']

today = datetime.now().strftime('%Y-%m-%d')
temp_df.to_excel(f'/Users/seanyu_mac/Downloads/아워박스_상품명_옵션명_업데이트_{today}_v1.xlsx')

In [53]:
temp_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 933 entries, 0 to 932
Data columns (total 54 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   index                933 non-null    int64  
 1   처리단계                 933 non-null    object 
 2   도서산간여부               933 non-null    object 
 3   고유번호                 933 non-null    float64
 4   합포장번호                930 non-null    float64
 5   주문서상태                933 non-null    object 
 6   주문번호                 933 non-null    object 
 7   상품주문번호               933 non-null    object 
 8   품목코드                 933 non-null    int64  
 9   업체품목코드               933 non-null    int64  
 10  품목추가정보               0 non-null      float64
 11  품목명                  933 non-null    object 
 12  채널상품명                930 non-null    object 
 13  옵션1                  930 non-null    object 
 14  채널명                  933 non-null    object 
 15  채널아이디                930 non-null    flo